# Praca domowa 4

## Wczytanie danych

In [1]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')

### Pierwsze dane

In [2]:
import dalex.datasets
df_apartments = dalex.datasets.load_apartments()
#df_apartments_test = dalex.datasets.load_apartments_test() też istnieje taki dataset, ale nie jest wtłumaczone jaka jest
#różnica, poza tym, że test ma 9000 wierszy a zwykły 1000.

In [3]:
df_apartments

,m2_price,construction_year,surface,floor,no_rooms,district
1,5897,1953,25,3,1,Srodmiescie
2,1818,1992,143,9,5,Bielany
3,3643,1937,56,1,2,Praga
4,3517,1995,93,7,3,Ochota
5,3013,1992,144,6,5,Mokotow
...,...,...,...,...,...,...
996,6355,1921,44,2,2,Srodmiescie
997,3422,1921,48,10,2,Bemowo
998,3098,1980,85,3,3,Bemowo
999,4192,1942,36,7,1,Zoliborz


In [4]:
from category_encoders import OneHotEncoder

encoder = OneHotEncoder(cols=['district'])
df_apartments = encoder.fit_transform(df_apartments, df_apartments['m2_price'])

In [5]:
from sklearn.model_selection import train_test_split
apartments_X, apartments_X_val, apartments_y, apartments_y_val = train_test_split(
    df_apartments.drop(columns=['m2_price']), df_apartments['m2_price'], test_size=0.2, random_state=2137)

### Drugie dane

In [6]:
import pandas as pd

In [7]:
#https://archive.ics.uci.edu/ml/datasets/Concrete+Compressive+Strength

xls = pd.ExcelFile("Concrete_Data.xls") 
df_concrete = xls.parse(0)

In [8]:
df_concrete

,Cement (component 1)(kg in a m^3 mixture),Blast Furnace Slag (component 2)(kg in a m^3 mixture),Fly Ash (component 3)(kg in a m^3 mixture),Water (component 4)(kg in a m^3 mixture),Superplasticizer (component 5)(kg in a m^3 mixture),Coarse Aggregate (component 6)(kg in a m^3 mixture),Fine Aggregate (component 7)(kg in a m^3 mixture),Age (day),"Concrete compressive strength(MPa, megapascals)"
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.986111
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.887366
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.269535
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.052780
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.296075
...,...,...,...,...,...,...,...,...,...
1025,276.4,116.0,90.3,179.6,8.9,870.1,768.3,28,44.284354
1026,322.2,0.0,115.6,196.0,10.4,817.9,813.4,28,31.178794
1027,148.5,139.4,108.6,192.7,6.1,892.4,780.0,28,23.696601
1028,159.1,186.7,0.0,175.6,11.3,989.6,788.9,28,32.768036


In [9]:
from sklearn.model_selection import train_test_split
concrete_X, concrete_X_val, concrete_y, concrete_y_val = train_test_split(
    df_concrete.drop(df_concrete.columns[8], axis=1), df_concrete.iloc[:, 8], test_size=0.2, random_state=2137)

## Dopasowanie modelu do apartments

In [10]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
svr1 = SVR()
svr1.fit(apartments_X, apartments_y)

print("Root Mean Squared Error : ", mean_squared_error(apartments_y_val, svr1.predict(apartments_X_val), squared=False))

Root Mean Squared Error :  909.9169754614527


## Dopasowanie modelu do concrete

In [11]:
svr2 = SVR()
svr2.fit(concrete_X, concrete_y)

print("Root Mean Squared Error : ", mean_squared_error(concrete_y_val, svr2.predict(concrete_X_val), squared=False))

Root Mean Squared Error :  14.041489819353407


## Czy warto skalować?

In [12]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df_apartments2 = df_apartments
df_apartments2[["construction_year", "surface", "floor", "no_rooms"]]=scaler.fit_transform(
    df_apartments[["construction_year", "surface", "floor", "no_rooms"]])
df_apartments2

,m2_price,construction_year,surface,floor,no_rooms,district_1,district_2,district_3,district_4,district_5,district_6,district_7,district_8,district_9,district_10
1,5897,0.366667,0.038462,0.222222,0.0,1,0,0,0,0,0,0,0,0,0
2,1818,0.800000,0.946154,0.888889,0.8,0,1,0,0,0,0,0,0,0,0
3,3643,0.188889,0.276923,0.000000,0.2,0,0,1,0,0,0,0,0,0,0
4,3517,0.833333,0.561538,0.666667,0.4,0,0,0,1,0,0,0,0,0,0
5,3013,0.800000,0.953846,0.555556,0.8,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,6355,0.011111,0.184615,0.111111,0.2,1,0,0,0,0,0,0,0,0,0
997,3422,0.011111,0.215385,1.000000,0.2,0,0,0,0,0,0,0,0,1,0
998,3098,0.666667,0.500000,0.222222,0.4,0,0,0,0,0,0,0,0,1,0
999,4192,0.244444,0.123077,0.666667,0.0,0,0,0,0,0,0,1,0,0,0


In [13]:
apartments2_X, apartments2_X_val, apartments2_y, apartments2_y_val = train_test_split(
    df_apartments2.drop(columns=['m2_price']), df_apartments2['m2_price'], test_size=0.2, random_state=2137)

In [14]:
svr3 = SVR()
svr3.fit(apartments2_X, apartments2_y)

print("Root Mean Squared Error : ", mean_squared_error(apartments2_y_val, svr3.predict(apartments2_X_val), squared=False))

Root Mean Squared Error :  885.443908679711


Widać więc, że warto skalować. Choć MSE nie zmniejszyło się znacznie, to jednak się zmniejszyło, przy innych random state'ach również ta zależność się powtarzała.

## Optymalizacja za pomocą random search

### Dla apartments

In [15]:
from sklearn.model_selection import RandomizedSearchCV

parameters = {
    'C': np.linspace(1, 10000, num=100),
    'gamma': ['auto', 'scale'],
    'degree': np.linspace(1, 1000, num=100)
}

clf_rand = RandomizedSearchCV(svr3, parameters, cv=3, n_iter=200)

clf_rand.fit(apartments2_X, apartments2_y)

RandomizedSearchCV(cv=3, estimator=SVR(), n_iter=200,
                   param_distributions={'C': array([1.000e+00, 1.020e+02, 2.030e+02, 3.040e+02, 4.050e+02, 5.060e+02,
       6.070e+02, 7.080e+02, 8.090e+02, 9.100e+02, 1.011e+03, 1.112e+03,
       1.213e+03, 1.314e+03, 1.415e+03, 1.516e+03, 1.617e+03, 1.718e+03,
       1.819e+03, 1.920e+03, 2.021e+03, 2.122e+03, 2.223e+03, 2.324e+03,
       2.425e+03, 2.526e+03, 2.627e+...
        727.54545455,  737.63636364,  747.72727273,  757.81818182,
        767.90909091,  778.        ,  788.09090909,  798.18181818,
        808.27272727,  818.36363636,  828.45454545,  838.54545455,
        848.63636364,  858.72727273,  868.81818182,  878.90909091,
        889.        ,  899.09090909,  909.18181818,  919.27272727,
        929.36363636,  939.45454545,  949.54545455,  959.63636364,
        969.72727273,  979.81818182,  989.90909091, 1000.        ]),
                                        'gamma': ['auto', 'scale']})

In [16]:
clf_rand.best_params_

{'gamma': 'scale', 'degree': 344.0909090909091, 'C': 1819.0}

In [17]:
print("Root Mean Squared Error : ", mean_squared_error(apartments2_y_val, clf_rand.best_estimator_
                                                  .predict(apartments2_X_val), squared=False))

Root Mean Squared Error :  166.45018082139183


### Dla concrete

In [18]:
clf_rand2 = RandomizedSearchCV(svr2, parameters, cv=3, n_iter=200)

clf_rand2.fit(concrete_X, concrete_y)

RandomizedSearchCV(cv=3, estimator=SVR(), n_iter=200,
                   param_distributions={'C': array([1.000e+00, 1.020e+02, 2.030e+02, 3.040e+02, 4.050e+02, 5.060e+02,
       6.070e+02, 7.080e+02, 8.090e+02, 9.100e+02, 1.011e+03, 1.112e+03,
       1.213e+03, 1.314e+03, 1.415e+03, 1.516e+03, 1.617e+03, 1.718e+03,
       1.819e+03, 1.920e+03, 2.021e+03, 2.122e+03, 2.223e+03, 2.324e+03,
       2.425e+03, 2.526e+03, 2.627e+...
        727.54545455,  737.63636364,  747.72727273,  757.81818182,
        767.90909091,  778.        ,  788.09090909,  798.18181818,
        808.27272727,  818.36363636,  828.45454545,  838.54545455,
        848.63636364,  858.72727273,  868.81818182,  878.90909091,
        889.        ,  899.09090909,  909.18181818,  919.27272727,
        929.36363636,  939.45454545,  949.54545455,  959.63636364,
        969.72727273,  979.81818182,  989.90909091, 1000.        ]),
                                        'gamma': ['auto', 'scale']})

In [19]:
clf_rand2.best_params_

{'gamma': 'scale', 'degree': 646.8181818181819, 'C': 10000.0}

In [20]:
print("Root Mean Squared Error : ", mean_squared_error(concrete_y_val, clf_rand2.best_estimator_
                                                  .predict(concrete_X_val), squared=False))

Root Mean Squared Error :  7.1029026163878575


### Wniosek

Hiperparametry znacząca zmniejszają błąd, czego można było się spodziewać. Jakbym wiedział, jak przełączyć jupyter na GPU, to bym chętnie nawet sprawdził dla znacznie większej ilości losowych parametrów.